# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
#import the necessary packages
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor()

In [3]:
# Your code here; create the connection

query = """
SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31'
);
"""

cursor.execute(query)

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [10]:
# Your cSELECT customerNumber, contactLastName, contactFirstName

query = """
SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31'
);
"""

cursor.execute(query)
results = cursor.fetchall()

# Print results
for row in results:
    print(row)


(141, 'Freyre', 'Diego ')


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [11]:
# Your code here
query = """
SELECT products.productName, COUNT(orderdetails.orderNumber) AS total_orders
FROM products
JOIN orderdetails ON products.productCode = orderdetails.productCode
GROUP BY products.productName
ORDER BY total_orders DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

# Print results
for row in results:
    print(row)

('1992 Ferrari 360 Spider red', 53)
('P-51-D Mustang', 28)
('HMS Bounty', 28)
('F/A 18 Hornet 1/72', 28)
('Diamond T620 Semi-Skirted Tanker', 28)
('Corsair F4U ( Bird Cage)', 28)
('Boeing X-32A JSF', 28)
('American Airlines: MD-11S', 28)
('American Airlines: B767-300', 28)
('America West Airlines B757-200', 28)
('ATA: B757-300', 28)
('2003 Harley-Davidson Eagle Drag Bike', 28)
('2002 Suzuki XREO', 28)
('1999 Yamaha Speed Boat', 28)
('1998 Chrysler Plymouth Prowler', 28)
('1997 BMW R 1100 S', 28)
('1997 BMW F650 ST', 28)
('1996 Peterbilt 379 Stake Bed with Outrigger', 28)
('1996 Moto Guzzi 1100i', 28)
('1982 Camaro Z28', 28)
('1980’s GM Manhattan Express', 28)
('1980s Black Hawk Helicopter', 28)
('1972 Alfa Romeo GTA', 28)
('1970 Dodge Coronet', 28)
('1969 Harley Davidson Ultimate Chopper', 28)
('1964 Mercedes Tour Bus', 28)
('1962 Volkswagen Microbus', 28)
('1962 LanciaA Delta 16V', 28)
('1960 BSA Gold Star DBD34', 28)
('1958 Setra Bus', 28)
('1958 Chevy Corvette Limited Edition', 28)


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [12]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
query = """
SELECT products.productName, COUNT(DISTINCT orders.customerNumber) AS total_customers
FROM products
JOIN orderdetails ON products.productCode = orderdetails.productCode
JOIN orders ON orderdetails.orderNumber = orders.orderNumber
GROUP BY products.productName
ORDER BY total_customers DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

('1992 Ferrari 360 Spider red', 40)
('Boeing X-32A JSF', 27)
('1972 Alfa Romeo GTA', 27)
('1952 Alpine Renault 1300', 27)
('1934 Ford V8 Coupe', 27)
('HMS Bounty', 26)
('Corsair F4U ( Bird Cage)', 26)
('American Airlines: MD-11S', 26)
('1999 Yamaha Speed Boat', 26)
('1996 Moto Guzzi 1100i', 26)
('1995 Honda Civic', 26)
('1980s Black Hawk Helicopter', 26)
('1969 Harley Davidson Ultimate Chopper', 26)
('1941 Chevrolet Special Deluxe Cabriolet', 26)
('1940 Ford Delivery Sedan', 26)
('1937 Horch 930V Limousine', 26)
('1913 Ford Model T Speedster', 26)
('1900s Vintage Bi-Plane', 26)
('18th Century Vintage Horse Carriage', 26)
('America West Airlines B757-200', 25)
('ATA: B757-300', 25)
('2003 Harley-Davidson Eagle Drag Bike', 25)
('2002 Suzuki XREO', 25)
('1997 BMW R 1100 S', 25)
('1993 Mazda RX-7', 25)
('1962 LanciaA Delta 16V', 25)
('1960 BSA Gold Star DBD34', 25)
('1948 Porsche 356-A Roadster', 25)
('1940s Ford truck', 25)
('1940 Ford Pickup Truck', 25)
('1937 Lincoln Berline', 25)
('193

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [14]:
# Your code here
query = """
SELECT DISTINCT employees.employeeNumber, employees.firstName, employees.lastName, offices.city, employees.officeCode
FROM employees
JOIN offices ON employees.officeCode = offices.officeCode
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
WHERE orderdetails.productCode IN (
    SELECT productCode
    FROM orderdetails
    GROUP BY productCode
    HAVING COUNT(DISTINCT orders.customerNumber) < 20
);
"""


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [16]:
# Your code here

query = """
SELECT employees.employeeNumber, employees.firstName, employees.lastName, COUNT(customers.customerNumber) AS total_customers
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
WHERE customers.customerNumber IN (
    SELECT customerNumber
    FROM customers
    GROUP BY salesRepEmployeeNumber
    HAVING AVG(creditLimit) > 15000
)
GROUP BY employees.employeeNumber
ORDER BY total_customers DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

(1702, 'Martin', 'Gerard', 1)
(1621, 'Mami', 'Nishi', 1)
(1612, 'Peter', 'Marsh', 1)
(1611, 'Andy', 'Fixter', 1)
(1504, 'Barry', 'Jones', 1)
(1501, 'Larry', 'Bott', 1)
(1401, 'Pamela', 'Castillo', 1)
(1370, 'Gerard', 'Hernandez', 1)
(1337, 'Loui', 'Bondur', 1)
(1323, 'George', 'Vanauf', 1)
(1286, 'Foon Yue', 'Tseng', 1)
(1216, 'Steve', 'Patterson', 1)
(1188, 'Julie', 'Firrelli', 1)
(1166, 'Leslie', 'Thompson', 1)
(1165, 'Leslie', 'Jennings', 1)


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!